In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import time
import re
import numpy as np
import pandas as pd
# import undetected_chromedriver.v2 as uc2
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import gc

In [2]:
df = pd.read_csv("smartphones_data_for_recommendation.csv")
df.head()

,name,link,price,rating,spec_score,processor_brand,processor_core,clock_speed,has_ir,has_nfc,...,foldable_display,display_refresh_rate,ppi,num_rear_cameras,num_front_cameras,rear_primary_mp,front_primary_mp,expandable_memory_supported,expandable_memory_limit,screen_type
0,Doogee S119,https://www.smartprix.com/mobiles/doogee-s119-...,29999.0,4.40,89,helio,octa,2.00,0,1,...,0,120.0,391.64,3,1,100.0,16.0,1,512.0,LCD
1,Realme Narzo 80x 5G (8GB RAM + 128GB),https://www.smartprix.com/mobiles/realme-narzo...,14499.0,4.05,80,dimensity,octa,2.50,0,0,...,0,120.0,391.64,2,1,50.0,8.0,1,1024.0,LCD
2,OnePlus 5,https://www.smartprix.com/mobiles/oneplus-5-p1...,22999.0,4.40,74,snapdragon,octa,2.45,0,1,...,0,120.0,400.53,2,1,20.0,16.0,0,0.0,AMOLED
3,Xiaomi Redmi Note 10 (6GB RAM + 128GB),https://www.smartprix.com/mobiles/xiaomi-redmi...,14999.0,4.50,78,snapdragon,octa,2.20,1,0,...,0,120.0,409.30,4,1,48.0,13.0,1,512.0,SUPER AMOLED
4,Realme Narzo 11,https://www.smartprix.com/mobiles/realme-narzo...,12999.0,4.35,75,helio,octa,2.20,0,0,...,0,120.0,265.84,4,1,50.0,16.0,1,256.0,LCD


In [9]:
driver = uc.Chrome()
driver.maximize_window()

link = "https://www.smartprix.com/mobiles/oppo-reno-2-ppd1ou11yxp0"
driver.get(link)
time.sleep(2)
img_tag = driver.find_element(By.XPATH, '//*[@id="app"]/main/div[1]/div[1]/div[1]/div[1]/div[1]/img')
image_url = img_tag.get_attribute("src")
print(image_url)
driver.quit()


https://cdn1.smartprix.com/rx-ijULh2d9g-w420-h420/oppo-reno-2.webp


In [3]:
df['link'].values

array(['https://www.smartprix.com/mobiles/doogee-s119-ppd1061ooffo',
       'https://www.smartprix.com/mobiles/realme-narzo-80x-5g-8gb-ram-128gb-ppd1nqmx4rvo',
       'https://www.smartprix.com/mobiles/oneplus-5-p1101pdglcaa', ...,
       'https://www.smartprix.com/mobiles/motorola-moto-g-go-ppd13u5bi8sz',
       'https://www.smartprix.com/mobiles/oppo-reno-2-ppd1ou11yxp0',
       'https://www.smartprix.com/mobiles/apple-iphone-14-plus-ppd1vn1wqlnc'],
      dtype=object)

In [12]:
image_paths = []

In [13]:
def start_browser():
    driver = uc.Chrome()
    driver.set_page_load_timeout(50)  # seconds
    driver.maximize_window()
    return driver

driver = start_browser()
i = 0
while i < df.shape[0]:
    if i % 10 == 0 and i != 0:
        print(f"Restarting browser at iteration {i}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()

    link = df.iloc[i]['link']
    
    try:
        driver.get(link)
    except TimeoutException:
        print(f"[{i}] Timeout while loading: {link}")
        image_paths.append(np.nan)
        i += 1
        continue
    except Exception as e:
        print(f"[{i}] Unexpected error loading page: {e}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()
        continue

    # image url info
    image_url = np.nan
    try:
        img_tag = driver.find_element(By.XPATH, '//*[@id="app"]/main/div[1]/div[1]/div[1]/div[1]/div[1]/img')
        image_url = img_tag.get_attribute("src")
    except NoSuchElementException:
        print(f"[{i}] image path info not found.")
    except Exception as e:
        print(f"[{i}] Error finding image path info: {e}")

    image_paths.append(image_url)

    print(f"[{i}] Scraped successfully.")
    time.sleep(1)
    i += 1

driver.quit()

[0] Scraped successfully.
[1] Scraped successfully.
[2] Scraped successfully.
[3] Scraped successfully.
[4] Scraped successfully.
[5] Scraped successfully.
[6] Scraped successfully.
[7] Scraped successfully.
[8] Scraped successfully.
[9] Scraped successfully.
Restarting browser at iteration 10
[10] Scraped successfully.
[11] Scraped successfully.
[12] Scraped successfully.
[13] Scraped successfully.
[14] Scraped successfully.
[15] Scraped successfully.
[16] Scraped successfully.
[17] Scraped successfully.
[18] Scraped successfully.
[19] Scraped successfully.
Restarting browser at iteration 20
[20] Scraped successfully.
[21] Scraped successfully.
[22] Scraped successfully.
[23] Scraped successfully.
[24] Scraped successfully.
[25] Scraped successfully.
[26] Scraped successfully.
[27] Scraped successfully.
[28] Scraped successfully.
[29] Scraped successfully.
Restarting browser at iteration 30
[30] Scraped successfully.
[31] Scraped successfully.
[32] Scraped successfully.
[33] Scraped s

URLError: <urlopen error [WinError 10051] A socket operation was attempted to an unreachable network>

In [15]:
len(image_paths)

1820

In [23]:
image_paths[-10:-1]

['https://cdn1.smartprix.com/rx-iIX8OBo69-w420-h420/realme-9-5g-se-8gb-r.webp',
 'https://cdn1.smartprix.com/rx-in2mMaLoI-w420-h420/vivo-v5-lite.webp',
 'https://cdn1.smartprix.com/rx-iSAvTwpiN-w420-h420/realme-11-5g-8gb-ram.webp',
 'https://cdn1.smartprix.com/rx-ihkwWBawe-w420-h420/poco-x5-pro-8gb-ram.webp',
 'https://cdn1.smartprix.com/rx-iwrPGN4P6-w420-h420/oppo-reno-13-5g-12gb.webp',
 nan,
 nan,
 nan,
 nan]

In [36]:
image_paths = image_paths[:1815]

In [54]:
# Due to some network problem, scraping stopped after fetching the image path of 1815th phone (index no. - 1814). So, again starting the web scraping from phone 1816 (index no. - 1815) onwards.

In [60]:
def start_browser():
    driver = uc.Chrome()
    driver.set_page_load_timeout(50)  # seconds
    driver.maximize_window()
    return driver

driver = start_browser()
i = 1815
while i < df.shape[0]:
    if i % 10 == 0 and i != 0:
        print(f"Restarting browser at iteration {i}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()

    link = df.iloc[i]['link']
    
    try:
        driver.get(link)
    except TimeoutException:
        print(f"[{i}] Timeout while loading: {link}")
        image_paths.append(np.nan)
        i += 1
        continue
    except Exception as e:
        print(f"[{i}] Unexpected error loading page: {e}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()
        continue

    # image url info
    image_url = np.nan
    try:
        img_tag = driver.find_element(By.XPATH, '//*[@id="app"]/main/div[1]/div[1]/div[1]/div[1]/div[1]/img')
        image_url = img_tag.get_attribute("src")
    except NoSuchElementException:
        print(f"[{i}] image path info not found.")
    except Exception as e:
        print(f"[{i}] Error finding image path info: {e}")

    image_paths.append(image_url)

    print(f"[{i}] Scraped successfully.")
    time.sleep(1)
    i += 1

driver.quit()

[1815] Scraped successfully.
[1816] Scraped successfully.
[1817] Scraped successfully.
[1818] Scraped successfully.
[1819] Scraped successfully.
Restarting browser at iteration 1820
[1820] Scraped successfully.
[1821] Scraped successfully.
[1822] Scraped successfully.
[1823] Scraped successfully.
[1824] Scraped successfully.
[1825] Scraped successfully.
[1826] Scraped successfully.
[1827] Scraped successfully.
[1828] Scraped successfully.
[1829] Scraped successfully.
Restarting browser at iteration 1830
[1830] Scraped successfully.
[1831] Scraped successfully.
[1832] Scraped successfully.
[1833] Scraped successfully.
[1834] Scraped successfully.
[1835] Scraped successfully.
[1836] Scraped successfully.
[1837] Scraped successfully.
[1838] Scraped successfully.
[1839] Scraped successfully.
Restarting browser at iteration 1840
[1840] Scraped successfully.
[1841] Scraped successfully.
[1842] Scraped successfully.
[1843] Scraped successfully.
[1844] Scraped successfully.
[1845] Scraped succ

KeyboardInterrupt: 

In [61]:
len(image_paths)

2330

In [62]:
image_paths[2329]

'https://cdn1.smartprix.com/rx-iCXtF8fFG-w420-h420/micromax-yu-yutopia.webp'

In [64]:
df.iloc[2329]['link']

'https://www.smartprix.com/mobiles/micromax-yu-yutopia-p1101ms3la2z'

In [68]:
driver = uc.Chrome()
driver.maximize_window()
url = "https://www.smartprix.com/mobiles/micromax-yu-yutopia-p1101ms3la2z"
driver.get(url)
time.sleep(2)
driver.quit()

KeyboardInterrupt: 

In [69]:
len(image_paths)

2330

In [ ]:
# Again, due to some network error, the web scraping was happening very slowly. So had to forcely stop it.
# Will try again later.

In [71]:
# But saving the current data to a temporary csv file first.

temp_df = pd.DataFrame({
    "image_path" : image_paths
})

temp_df.to_csv("incomplete_image_paths.csv", index = False)

In [3]:
temp_df = pd.read_csv("incomplete_image_paths.csv")
temp_df.head()

,image_path
0,https://cdn1.smartprix.com/rx-iDLrTC6Js-w420-h...
1,https://cdn1.smartprix.com/rx-iMN81O6Eq-w420-h...
2,https://cdn1.smartprix.com/rx-iMRK2tPAg-w420-h...
3,https://cdn1.smartprix.com/rx-iOuwWGdpp-w420-h...
4,https://cdn1.smartprix.com/rx-iRcwwMVqi-w420-h...


In [6]:
temp_df.shape

(2330, 1)

In [12]:
temp_df.iloc[2328]['image_path']

'https://cdn1.smartprix.com/rx-iJmKCLd2L-w420-h420/poco-m6-5g.webp'

In [13]:
image_paths = temp_df['image_path'].to_list()

In [15]:
image_paths[::-1]

['https://cdn1.smartprix.com/rx-iCXtF8fFG-w420-h420/micromax-yu-yutopia.webp',
 'https://cdn1.smartprix.com/rx-iJmKCLd2L-w420-h420/poco-m6-5g.webp',
 'https://cdn1.smartprix.com/rx-iyvAiCwtn-w420-h420/itel-p40.webp',
 'https://cdn1.smartprix.com/rx-igRGKFTyB-w420-h420/xiaomi-redmi-10x-pro.webp',
 'https://cdn1.smartprix.com/rx-ikNbpIFBh-w420-h420/honor-x10-max-5g.webp',
 'https://cdn1.smartprix.com/rx-iYUP0R45O-w420-h420/vivo-x90-pro-plus-5g.webp',
 'https://cdn1.smartprix.com/rx-ieDRb75up-w420-h420/samsung-galaxy-on-ma.webp',
 'https://cdn1.smartprix.com/rx-i2lL8thRG-w420-h420/samsung-galaxy-s23-f.webp',
 'https://cdn1.smartprix.com/rx-iFFm2z8tM-w420-h420/lava-blaze-curve-5g.webp',
 'https://cdn1.smartprix.com/rx-ix4fyNsHn-w420-h420/huawei-p-smart-2021.webp',
 'https://cdn1.smartprix.com/rx-iIFxjMyka-w420-h420/xiaomi-redmi-6a-2gb.webp',
 'https://cdn1.smartprix.com/rx-iVQKxANAX-w420-h420/vivo-y52-5g.webp',
 'https://cdn1.smartprix.com/rx-iL4IeoAwV-w420-h420/motorola-moto-g7.webp',
 'h

In [18]:
df.iloc[2325:2330]

,name,link,price,rating,spec_score,processor_brand,processor_core,clock_speed,has_ir,has_nfc,...,foldable_display,display_refresh_rate,ppi,num_rear_cameras,num_front_cameras,rear_primary_mp,front_primary_mp,expandable_memory_supported,expandable_memory_limit,screen_type
2325,Honor X10 Max 5G,https://www.smartprix.com/mobiles/honor-x10-ma...,19990.0,4.15,78,dimensity,octa,2.0,0,1,...,0,90.0,355.83,3,1,48.0,8.0,1,256.0,LCD
2326,Xiaomi Redmi 10X Pro 5G (8GB RAM + 256GB),https://www.smartprix.com/mobiles/xiaomi-redmi...,26999.0,4.25,88,dimensity,octa,2.6,1,1,...,0,120.0,400.58,4,1,48.0,20.0,1,256.0,AMOLED
2327,Itel P40,https://www.smartprix.com/mobiles/itel-p40-ppd...,6199.0,4.00,61,other,octa,1.6,0,0,...,0,90.0,267.50,2,1,13.0,5.0,1,512.0,LCD
2328,Poco M6 5G,https://www.smartprix.com/mobiles/poco-m6-ppd1...,8499.0,4.75,73,dimensity,octa,2.2,1,0,...,0,90.0,267.10,2,1,50.0,5.0,1,1024.0,LCD
2329,Micromax Yu Yutopia,https://www.smartprix.com/mobiles/micromax-yu-...,7310.0,4.25,62,snapdragon,octa,1.5,0,0,...,0,90.0,564.85,1,1,21.0,8.0,1,128.0,LCD


In [19]:
def start_browser():
    driver = uc.Chrome()
    driver.set_page_load_timeout(50)  # seconds
    driver.maximize_window()
    return driver

driver = start_browser()
i = 2330
while i < df.shape[0]:
    if i % 10 == 0 and i != 0:
        print(f"Restarting browser at iteration {i}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()

    link = df.iloc[i]['link']
    
    try:
        driver.get(link)
    except TimeoutException:
        print(f"[{i}] Timeout while loading: {link}")
        image_paths.append(np.nan)
        i += 1
        continue
    except Exception as e:
        print(f"[{i}] Unexpected error loading page: {e}")
        driver.quit()
        time.sleep(2)
        driver = start_browser()
        gc.collect()
        continue

    # image url info
    image_url = np.nan
    try:
        img_tag = driver.find_element(By.XPATH, '//*[@id="app"]/main/div[1]/div[1]/div[1]/div[1]/div[1]/img')
        image_url = img_tag.get_attribute("src")
    except NoSuchElementException:
        print(f"[{i}] image path info not found.")
    except Exception as e:
        print(f"[{i}] Error finding image path info: {e}")

    image_paths.append(image_url)

    print(f"[{i}] Scraped successfully.")
    time.sleep(1)
    i += 1

driver.quit()

Restarting browser at iteration 2330
[2330] Scraped successfully.
[2331] Scraped successfully.
[2332] Scraped successfully.
[2333] Scraped successfully.
[2334] Scraped successfully.
[2335] Scraped successfully.
[2336] Scraped successfully.
[2337] Scraped successfully.
[2338] Scraped successfully.
[2339] Scraped successfully.
Restarting browser at iteration 2340
[2340] Scraped successfully.
[2341] Scraped successfully.
[2342] Scraped successfully.
[2343] Scraped successfully.
[2344] Scraped successfully.
[2345] Scraped successfully.
[2346] Scraped successfully.
[2347] Scraped successfully.
[2348] Scraped successfully.
[2349] Scraped successfully.
Restarting browser at iteration 2350
[2350] Scraped successfully.
[2351] Scraped successfully.
[2352] Scraped successfully.
[2353] Scraped successfully.
[2354] Scraped successfully.
[2355] Scraped successfully.
[2356] Scraped successfully.
[2357] Scraped successfully.
[2358] Scraped successfully.
[2359] Scraped successfully.
Restarting browser 

In [20]:
len(image_paths)

4058

In [22]:
df['image_path'] = image_paths

In [25]:
df[['name', 'image_path']].values

array([['Doogee S119',
        'https://cdn1.smartprix.com/rx-iDLrTC6Js-w420-h420/doogee-s119.webp'],
       ['Realme Narzo 80x 5G (8GB RAM + 128GB)',
        'https://cdn1.smartprix.com/rx-iMN81O6Eq-w420-h420/realme-narzo-80x-5g.webp'],
       ['OnePlus 5',
        'https://cdn1.smartprix.com/rx-iMRK2tPAg-w420-h420/oneplus-5.webp'],
       ...,
       ['Motorola Moto G Go',
        'https://cdn1.smartprix.com/rx-iXYvaV3Pu-w420-h420/motorola-moto-g-go.webp'],
       ['OPPO Reno 2',
        'https://cdn1.smartprix.com/rx-ijULh2d9g-w420-h420/oppo-reno-2.webp'],
       ['Apple iPhone 14 Plus',
        'https://cdn1.smartprix.com/rx-ibQpWfEfh-w420-h420/apple-iphone-14-plus.webp']],
      dtype=object)

In [26]:
df.isnull().sum()

name                           0
link                           0
price                          0
rating                         0
spec_score                     0
processor_brand                0
processor_core                 0
clock_speed                    0
has_ir                         0
has_nfc                        0
has_5g                         0
ram_gb                         0
rom_gb                         0
brand                          0
battery_capacity_mah           0
fast_charging_watt             0
screen_size_inch               0
foldable_display               0
display_refresh_rate           0
ppi                            0
num_rear_cameras               0
num_front_cameras              0
rear_primary_mp                0
front_primary_mp               0
expandable_memory_supported    0
expandable_memory_limit        0
screen_type                    0
image_path                     2
dtype: int64

In [27]:
df.dropna(inplace = True)

In [29]:
# Exporting the dataframe
df.to_csv("phones_with_image_path.csv", index = False)